In [1]:
from z3 import *

# I. Задача 1

Рещим систему уравнений<br>

x * y = 12<br>
x + y = 7<br>
x > 0<br>
y > 0<br>

Тогда цель Goal будет нйти значения x и y

In [35]:
x, y = Reals('x y')
g  = Goal()
g.add(x > 0, y > 0, x * y == 12, x + 2 * y == 7 + y)

Добавим тактики упрощения и тактику для нелинейной арифметики

In [ ]:
# Применить стратегию к Goal напрямую
t1 = Tactic('purify-arith')
t2 = Tactic('nlsat')
t = Then(t1, t2)

s = t.solver()
s.add(g)

if s.check() == sat:
    m = s.model()
    print(f"\nSolution:")
    print(f"x = {m.evaluate(x)}")
    print(f"y = {m.evaluate(y)}")



Solution:
x = 3
y = 4


# II. Задача 2
Ограничения:<br>
<br>
(a or b)<br>
(not a or c)<br>
(not b or c)<br>
<br>
Цель - определить верна ли система выражений<br>
Формула полностью логическая и может быть разделена на независимые подцели.<br>
<br>
Используем тактики:<br>
simplify<br>
split-clause<br>

In [45]:

a, b, c = Bools('a b c')
g  = Goal()
g.add(Or(a, b),
      Or(Not(a), c),
      Or(Not(b), c))

t_simplify = Tactic('simplify')
t_split = Tactic('split-clause')
t_sat = Tactic('sat')

t = Then(t_simplify, t_split, t_sat)

s = t.solver()
s.add(g)
s.check()
m = s.model()
print(f"\nSolution:")
print(f"a = {m.evaluate(a)}")
print(f"b = {m.evaluate(b)}")
print(f"c = {m.evaluate(c)}")


Solution:
a = True
b = False
c = True


# III Задача 3

### Поиск Оптимального Маршрута с Логическими Ограничениями
#### Описание задачи
У вас есть сеть дорог между 4 городами (A, B, C, D). Нужно найти маршрут из города A в город D, удовлетворяющий следующим условиям:

- Маршрут должен существовать (булевы переменные для рёбер)

Логические ограничения:
- Либо пройти через B, либо через C (но не через оба сразу)
- Если проходим через B, то обязательно нужно избежать дороги B→D напрямую (слишком опасно)

Арифметические ограничения:
- Общая длина маршрута не должна превышать 15 км
- Если маршрут содержит > 2 дорог, то общая стоимость ≤ 20 единиц

Данные о дорогах
| Дорога | Длина (км) | Стоимость |
|--------|------------|-----------|
| A→B    |         5  |        3  |
| A→C    |         8  |        4  |
| B→C    |         3  |        2  |
| B→D    |         6  |        5  |
| C→D    |         4  |        3  |


In [12]:
# Булевы переменные для каждого ребра (есть ли в маршруте)
a_b = Bool('A_B')
a_c = Bool('A_C')
b_c = Bool('B_C')
b_d = Bool('B_D')
c_d = Bool('C_D')

# Булевы переменные посещения городов
visit_A = True
visit_B = Bool('visit_B')
visit_C = Bool('visit_C')
visit_D = Bool('visit_D')

# Арифметические переменные
total_length = Int('total_length')
total_cost = Int('total_cost')
num_edges = Int('num_edges')

# Данные о дорогах
lengths = {
    'A_B': 5, 'A_C': 8, 'B_C': 3, 'B_D': 6, 'C_D': 4
}
costs = {
    'A_B': 3, 'A_C': 4, 'B_C': 2, 'B_D': 5, 'C_D': 3
}

constraints = []
constraints.append(visit_A)

# 1. Связь булевых переменных посещения с рёбрами
constraints.append(visit_B == Or(a_b, b_c))
constraints.append(visit_C == Or(a_c, b_c))
constraints.append(visit_D == Or(b_d, c_d))
constraints.append(visit_D)

# 2. БУЛЕВОЕ ОГРАНИЧЕНИЕ: Либо B, либо C (но не оба)
constraints.append(Xor(visit_B, visit_C))

# 3. БУЛЕВОЕ ОГРАНИЧЕНИЕ: Если посещаем B, то не идём напрямую B→D
constraints.append(Implies(visit_B, Not(b_d)))

# 4. Логика маршрута: начинаем из A, заканчиваем в D
# Вариант 1: A→B→C→D
constraints.append(If(And(a_b, b_c, c_d),
                      And(Not(a_c), Not(b_d)),
                      True))

# Вариант 2: A→B→D (но блокирован логикой выше)
# Вариант 3: A→C→D
constraints.append(If(And(a_c, c_d),
                      And(Not(a_b), Not(b_c), Not(b_d)),
                      True))

# 5. Вычисление общей длины и стоимости
constraints.append(total_length == 
    If(a_b, lengths['A_B'], 0) +
    If(a_c, lengths['A_C'], 0) +
    If(b_c, lengths['B_C'], 0) +
    If(b_d, lengths['B_D'], 0) +
    If(c_d, lengths['C_D'], 0))

constraints.append(total_cost ==
    If(a_b, costs['A_B'], 0) +
    If(a_c, costs['A_C'], 0) +
    If(b_c, costs['B_C'], 0) +
    If(b_d, costs['B_D'], 0) +
    If(c_d, costs['C_D'], 0))

constraints.append(num_edges ==
    If(a_b, 1, 0) +
    If(a_c, 1, 0) +
    If(b_c, 1, 0) +
    If(b_d, 1, 0) +
    If(c_d, 1, 0))

# 6. Арифметические ограничения
constraints.append(total_length <= 15)
constraints.append(Implies(num_edges > 2, total_cost <= 20))

# Стратегия с комбинаторами тактик
# 1. simplify - упростить булевы выражения
# 2. split-clause - разобрать Or/Xor выражения на подцели
# 3. solve-eqs - исключить переменные методом Гаусса
# 4. sat - классический SAT-решатель для булевой части

strategy = Then(
    With('simplify', arith_lhs=True),
    Repeat(OrElse(Tactic('split-clause'), Tactic('skip'))),
    Tactic('solve-eqs'),
    Tactic('sat')
)

solver = strategy.solver()
solver.add(constraints)

if solver.check() == sat:
    model = solver.model()
    print("Используемые дороги:")
    if model.evaluate(a_b):
        print(f"A → B: {lengths['A_B']} км, {costs['A_B']} единиц")
    if model.evaluate(a_c):
        print(f"A → C: {lengths['A_C']} км, {costs['A_C']} единиц")
    if model.evaluate(b_c):
        print(f"B → C: {lengths['B_C']} км, {costs['B_C']} единиц")
    if model.evaluate(b_d):
        print(f"B → D: {lengths['B_D']} км, {costs['B_D']} единиц")
    if model.evaluate(c_d):
        print(f"C → D: {lengths['C_D']} км, {costs['C_D']} единиц")

    print(f"Общая длина: {model.evaluate(total_length)} км")
    print(f"Общая стоимость: {model.evaluate(total_cost)} единиц")
    print(f"Количество дорог: {model.evaluate(num_edges)}")
    print(f"Посещаем B: {model.evaluate(visit_B)}")
    print(f"Посещаем C: {model.evaluate(visit_C)}")
    
    print(f"Длина ≤ 15: {model.evaluate(total_length) <= 15}")
    print(f"(Рёбер > 2) → (Стоимость ≤ 20): OK")

else:
    print("Решение не найдено!")

Используемые дороги:
A → C: 8 км, 4 единиц
C → D: 4 км, 3 единиц
Общая длина: 12 км
Общая стоимость: 7 единиц
Количество дорог: 2
Посещаем B: False
Посещаем C: True
Длина ≤ 15: 12 <= 15
(Рёбер > 2) → (Стоимость ≤ 20): OK


# IV Задача 4

### Система нелинейных уравнений для расчёта химической реакции:
  x² + y² = 25<br>
  x*y = 12<br>
  x > 0, y > 0<br>

Применим последовательно разные тактики:
1. simplify - упростить выражения
2. solve-eqs - решить уравнения относительно переменных
3. split-clause - разбить на подзадачи

In [46]:
x, y = Reals('x y')

goal = Goal()
goal.add(x*x + y*y == 25)
goal.add(x*y == 12)
goal.add(x > 0)
goal.add(y > 0)

# t_simplify = Tactic('simplify')
# result1 = t_simplify(goal)
# print(f"Результат: {len(result1)} подцель(ей)")
# print(result1[0])

t_combined = Then(Tactic('simplify'), Tactic('solve-eqs'), Tactic('smt'))

solver = t_combined.solver()
solver.add(goal.as_expr())

if solver.check() == sat:
    m = solver.model()
    print(f"Решение: x = {m[x]}, y = {m[y]}")


Решение: x = 4, y = 3


# V Задача 5
### Разрешение зависимостей пакетов
#### Менеджер пакетов должен установить 5 библиотек так, чтобы их версии были совместимы.

Библиотеки и доступные версии:
- Django: 3.2, 4.0, 4.1, 4.2
- Celery: 5.0, 5.1, 5.2, 5.3
- Redis: 3.5, 4.0, 4.5, 5.0
- PostgreSQL-adapter: 2.8, 2.9, 3.0, 3.1
- NumPy: 1.21, 1.22, 1.23, 1.24

Правила совместимости (несовместимые комбинации):
- Django 3.2 несовместима с Celery 5.3
- Django 4.2 требует Celery >= 5.2
- Django 4.x требует PostgreSQL >= 2.9
- Django 3.2 несовместима с PostgreSQL 3.1
- Celery 5.0-5.1 работают только с Redis < 5.0
- Celery 5.3 требует Redis >= 4.5
- PostgreSQL 3.0+ несовместим с NumPy 1.21
- PostgreSQL 2.8 требует NumPy <= 1.22
- Django 4.2 требует NumPy >= 1.23
- Redis 5.0 несовместим с NumPy 1.21-1.22

In [48]:
# Определяем версии как целые числа для простоты (индексы в списке версий)
django = Int('django')
celery = Int('celery')
redis = Int('redis')
postgres = Int('postgres')
numpy = Int('numpy')

# Словари для человекочитаемого вывода
versions = {
    'django': ['3.2', '4.0', '4.1', '4.2'],
    'celery': ['5.0', '5.1', '5.2', '5.3'],
    'redis': ['3.5', '4.0', '4.5', '5.0'],
    'postgres': ['2.8', '2.9', '3.0', '3.1'],
    'numpy': ['1.21', '1.22', '1.23', '1.24']
}

# Индексы версий для удобства
DJANGO_32, DJANGO_40, DJANGO_41, DJANGO_42 = 0, 1, 2, 3
CELERY_50, CELERY_51, CELERY_52, CELERY_53 = 0, 1, 2, 3
REDIS_35, REDIS_40, REDIS_45, REDIS_50 = 0, 1, 2, 3
PG_28, PG_29, PG_30, PG_31 = 0, 1, 2, 3
NUMPY_121, NUMPY_122, NUMPY_123, NUMPY_124 = 0, 1, 2, 3

# Создаем Goal с ограничениями
goal = Goal()

# 1. Каждая библиотека должна иметь одну из доступных версий
goal.add(And(django >= 0, django <= 3))
goal.add(And(celery >= 0, celery <= 3))
goal.add(And(redis >= 0, redis <= 3))
goal.add(And(postgres >= 0, postgres <= 3))
goal.add(And(numpy >= 0, numpy <= 3))

# 2. Django-Celery несовместимости
goal.add(Implies(django == DJANGO_32, celery != CELERY_53))
goal.add(Implies(django == DJANGO_42, celery >= CELERY_52))

# 3. Django-PostgreSQL несовместимости
goal.add(Implies(django >= DJANGO_40, postgres >= PG_29))
goal.add(Implies(django == DJANGO_32, postgres != PG_31))

# 4. Celery-Redis несовместимости
goal.add(Implies(celery <= CELERY_51, redis < REDIS_50))
goal.add(Implies(celery == CELERY_53, redis >= REDIS_45))

# 5. PostgreSQL-NumPy несовместимости
goal.add(Implies(postgres >= PG_30, numpy != NUMPY_121))
goal.add(Implies(postgres == PG_28, numpy <= NUMPY_122))

# 6. Django-NumPy несовместимости
goal.add(Implies(django == DJANGO_42, numpy >= NUMPY_123))

# 7. Redis-NumPy несовместимости
goal.add(Implies(redis == REDIS_50, numpy >= NUMPY_123))

# Функция для красивого вывода результата
def print_solution(model, title):
    print(f"\n{title}")
    print("-" * 60)
    if model is None:
        print("✗ Решение не найдено (несовместимые зависимости)")
        return False
    
    print("✓ Совместимый набор найден:\n")
    libs = [
        ('Django', django, 'django'),
        ('Celery', celery, 'celery'),
        ('Redis', redis, 'redis'),
        ('PostgreSQL', postgres, 'postgres'),
        ('NumPy', numpy, 'numpy')
    ]
    
    for name, var, key in libs:
        version_idx = model[var].as_long()
        version_str = versions[key][version_idx]
        print(f"  {name:15} → версия {version_str}")
    return True

# Стратегия разбиение на случаи: simplify → split-clause → propagate-values
strategy = Then(
    Tactic('simplify'),
    Tactic('split-clause'),
    Tactic('propagate-values')
)
result = strategy(goal)

solver = Solver()
for subgoal in result:
    for constraint in subgoal:
        solver.add(constraint)

if solver.check() == sat:
    print_solution(solver.model(), "Результат:")
else:
    print_solution(None, "Результат:")



Результат:
------------------------------------------------------------
✓ Совместимый набор найден:

  Django          → версия 4.0
  Celery          → версия 5.0
  Redis           → версия 3.5
  PostgreSQL      → версия 2.9
  NumPy           → версия 1.22


# ChatGPT
Общее впечатление от использования Чата для решения этих задач - он, во-первых, решает их похожим способом, в том смысле, что при сформулированных ограничениях он старается их применить (иногда в лоб) и чем больше ограничений (если они не противоречат друг другу) - тем качественнее решение, а во-вторых, при просьбе решить эти задачи через Z3 он не применяет тактик, а сливает все сразу в стандартный солвер. Возможно, конечно, что эти задачи для тактик и стратегий слишком примитивные. Иногда он решает неправильно, в том смысле, что его код просто не работает и наводящими вопросами и указаниями не всегда получается добиться от него работающих улучшений. Но в целом, чаще всего такие учебные задачи он решает успешно.